# Read spreadsheets from David and update database
These spreadsheets were imported on February 2022, I need to update the database.
## load libraries and DB connection parameters

In [3]:
import openpyxl
from pathlib import Path
import os

In [4]:
repodir = Path("../") 
inputdir = repodir / "data/"

Database credentials are stored in a database.ini file

In [5]:
from configparser import ConfigParser
import psycopg2

filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

    

## Read spreadsheets
Each spreadsheet has a slightly different structure, so these scripts have to be adapted for each case.
### Newnes Plateau Shrub Swamp

In [6]:
inputfile=inputdir / "field-form" / "Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm"
# using data_only=True to get the calculated cell values
wb = openpyxl.load_workbook(inputfile,data_only=True)

In [7]:
ws = wb['BS1']

In [8]:
ws['A1'].value

'Site information'

In [9]:
ws.cell(row=29, column=1).value

'Quadrat #'

## List of sites visited

In [30]:
ws = wb['Site']

In [31]:
ws

<Worksheet "Site">

In [70]:
row_count = ws.max_row
j=1
unique_list = list()
unique_items = 0

SSID = 28356 # assuming all coords in UTM zone 56
qrystr = "INSERT INTO form.field_site(site_label,geom,elevation) values('%s',ST_GeomFromText('POINT(%s %s)', %s),%s) ON CONFLICT DO NOTHING"
qryvst = "INSERT INTO form.field_visit(survey_name,visit_id,visit_date,createdbysystemuserid,datecreated,updatedbysystemuserid,dateupdated) values(%s,%s,%s,1,CURRENT_TIMESTAMP(0),1,CURRENT_TIMESTAMP(0)) ON CONFLICT (visit_id,visit_date) DO UPDATE SET survey_name=EXCLUDED.survey_name "

survey="NEWNES"
# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

ws.cell(row=1, column=j).value
for i in range(2, row_count + 1):
    item = ws.cell(row=i, column=j).value
    visitdate = ws.cell(row=i, column=9).value
    if item not in unique_list and item is not None:
        unique_list.append(item)
        unique_items += 1
        x=ws.cell(row=i,column=2).value
        y=ws.cell(row=i,column=3).value
        z=ws.cell(row=i,column=5).value
        cur.execute(qrystr % (item,x,y,SSID,z))
        updated_rows = cur.rowcount
        if updated_rows > 0:
            print("%s : %s rows updated" % (item,updated_rows))
    if isinstance(visitdate, datetime):
        cur.execute(qryvst, (survey,item,visitdate.date()))
        updated_rows = cur.rowcount
        if updated_rows > 0:
            print("%s : %s rows updated" % (item,updated_rows))
        
conn.commit()        
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Connecting to the PostgreSQL database...
BS1 : 1 rows updated
BS2 : 1 rows updated
MW1 : 1 rows updated
MW2 : 1 rows updated
HV1 : 1 rows updated
HV2 : 1 rows updated
SS1 : 1 rows updated
SS2 : 1 rows updated
GGE1 : 1 rows updated
GGE2 : 1 rows updated
GGW1 : 1 rows updated
GGW2 : 1 rows updated
CW1 : 1 rows updated
CW2 : 1 rows updated
CC1 : 1 rows updated
CC2 : 1 rows updated
EW1 : 1 rows updated
EW2 : 1 rows updated
BUD1 : 1 rows updated
BUD2 : 1 rows updated
BS1 : 1 rows updated
BS2 : 1 rows updated
MW1 : 1 rows updated
MW2 : 1 rows updated
HV1 : 1 rows updated
HV2 : 1 rows updated
SS1 : 1 rows updated
SS2 : 1 rows updated
GGE1 : 1 rows updated
GGE2 : 1 rows updated
GGW1 : 1 rows updated
GGW2 : 1 rows updated
CW1 : 1 rows updated
CW2 : 1 rows updated
CC1 : 1 rows updated
CC2 : 1 rows updated
EW1 : 1 rows updated
EW2 : 1 rows updated
BUD1 : 1 rows updated
BUD2 : 1 rows updated
MW1 : 1 rows updated
MW2 : 1 rows updated
HV1 : 1 rows updated
HV2 : 1 rows updated
SS1 : 1 rows updated
SS

## quadrat samples

In [10]:
ws = wb['Floristics']

In [101]:
k=2364
(ws.cell(row=k, column=1).value,
 ws.cell(row=k, column=2).value,
 ws.cell(row=k, column=5).value,
 ws.cell(row=k, column=6).value,
 ws.cell(row=k, column=7).value,
 ws.cell(row=k, column=8).value,
 ws.cell(row=k, column=9).value,
 ws.cell(row=k, column=10).value,
 ws.cell(row=k, column=11).value,
 ws.cell(row=k, column=12).value,
 ws.cell(row=k, column=13).value,
 ws.cell(row=k, column=14).value,
 ws.cell(row=k, column=15).value,
 ws.cell(row=k, column=16).value,
 ws.cell(row=k, column=17).value)
 


('BS2',
 1,
 datetime.datetime(2020, 11, 17, 0, 0),
 'Empodisma minus',
 5532,
 None,
 None,
 None,
 16,
 0,
 'na',
 0,
 'na',
 'na',
 'na')

In [98]:
ws[2088][2].value
#ws[578][11].value == 'NA'


'BS1.1'

In [110]:
from datetime import datetime
from psycopg2.extensions import AsIs

row_min = 3
row_max = ws.max_row

#row_min = 3001
#row_max = row_max
valid_organ=('epicormic', 'apical', 'lignotuber', 'basal','tuber','tussock','short rhizome', 'long rhizome or root sucker', 'stolon', 'none', 'other')
valid_seedbank=('soil-persistent', 'transient', 'canopy','non-canopy','other')
# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

for i in range(row_min, row_max):
    item = ws[i]
    species=item[5].value
    spcode=item[6].value
    visit_date = item[4].value 
    if species is not None and isinstance(visit_date,datetime) :
        record={'visit_id': item[0].value, 'sample_nr': item[1].value,
                'visit_date' : visit_date.date()}
        insert_statement = 'insert into form.field_samples (%s) values %s ON CONFLICT DO NOTHING'
        cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))
        record['species']=species
        if (isinstance(spcode, str) and spcode.isnumeric()) or isinstance(spcode,int):
            record['species_code']=spcode
        if item[7].value is not None and item[7].value in valid_organ:
            record['resprout_organ']=item[7].value
        if item[8].value is not None and item[8].value in valid_seedbank:
            record['seedbank']=item[8].value
        if item[9].value is not None and item[9].value not in ('na','NA'):
            record['adults_unburnt']=item[9].value
        if item[10].value is not None and item[10].value not in ('na','NA'):
            record['resprouts_live']=item[10].value
        if item[11].value is not None and item[11].value not in ('na','NA'):
            record['resprouts_died']=item[11].value
        if item[12].value is not None and item[12].value not in ('na','NA'):
            record['resprouts_kill']=item[12].value
        if item[13].value is not None and item[13].value not in ('na','NA'):
            record['resprouts_reproductive']=item[13].value
        if item[14].value is not None and item[14].value not in ('na','NA'):
            record['recruits_live']=item[14].value
        if item[15].value is not None and item[15].value not in ('na','NA'):
            record['recruits_died']=item[15].value
        if item[16].value is not None and item[16].value not in ('na','NA'):
            record['recruits_reproductive']=item[16].value
        if item[17].value is not None and item[17].value not in ('na','NA'):
            record['species_notes']=item[17].value
        #print(','.join(record.keys()))
        #print(record.values())
        insert_statement = 'insert into form.quadrat_samples (%s) values %s ON CONFLICT DO NOTHING'
        #print(i)
        #print(cur.mogrify(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values()))))
        cur.execute(insert_statement, (AsIs(','.join(record.keys())), tuple(record.values())))

conn.commit()        
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Connecting to the PostgreSQL database...
Database connection closed.


In [109]:
print(row_max)

5237


In [ ]:
row_count = ws.max_row

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)
cur = conn.cursor()

for i in range(3, row_count):
    item = ws[i]
    site_label = item[0].value 
    sample_nr = item[1].value 
    visit_date = item[4].value 
    if is_instance(visit_date,datetime) :
        
        unique_list.append(item)
        unique_items += 1
        x=ws.cell(row=i,column=2).value
        y=ws.cell(row=i,column=3).value
        z=ws.cell(row=i,column=5).value
        cur.execute(qrystr % (item,x,y,SSID,z))
        updated_rows = cur.rowcount
        if updated_rows > 0:
            print("%s : %s rows updated" % (item,updated_rows))
    if isinstance(visitdate, datetime):
        cur.execute(qryvst, (survey,item,visitdate.date()))
        updated_rows = cur.rowcount
        if updated_rows > 0:
            print("%s : %s rows updated" % (item,updated_rows))
        
conn.commit()        
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')